### <div class="alert alert-block alert-warning">There are some interactive plot. Please wait loading⏳</div>

In this notebook, I'll try simple visualization with plotly.

Now, I visualize following 2 data, 

- CONVENIENT_global_confirmed_cases.csv 

- CONVENIENT_global_deaths.csv

And I try to visualize trend of their data.

# Load library and check data

First, I load library. And check data I use. To visualize, I do some preprocess.

In [ ]:
from datetime import datetime as dt
import warnings

import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import statsmodels.api as sm

warnings.filterwarnings('ignore')

In [ ]:
!ls ../input/covid19-data-from-john-hopkins-university

In [ ]:
df_global_confirmed_cases = pd.read_csv("../input/covid19-data-from-john-hopkins-university/CONVENIENT_global_confirmed_cases.csv")
df_global_confirmed_cases = df_global_confirmed_cases[1:]
df_global_confirmed_cases = df_global_confirmed_cases.rename(columns={"Country/Region": "Date"})

def convert_date(date_str):
    return dt.strptime(date_str, '%m/%d/%y')

df_global_confirmed_cases["Date"] = df_global_confirmed_cases["Date"].map(convert_date)

country_cols = [col for col in df_global_confirmed_cases.columns if col not in ["Date"]]

df_global_confirmed_cases.head()

In [ ]:
df_global_deaths = pd.read_csv("../input/covid19-data-from-john-hopkins-university/CONVENIENT_global_deaths.csv")
df_global_deaths = df_global_deaths[1:]
df_global_deaths = df_global_deaths.rename(columns={"Country/Region": "Date"})

def convert_date(date_str):
    return dt.strptime(date_str, '%m/%d/%y')

df_global_deaths["Date"] = df_global_deaths["Date"].map(convert_date)

df_global_deaths.head()

# global_confirmed_cases

### tyding

In [ ]:
df_global_confirmed_cases = pd.melt(df_global_confirmed_cases, id_vars=['Date'], value_vars=country_cols,
                                    var_name='Country', value_name='Count')

In [ ]:
df_global_confirmed_cases.head()

### Visualize all data

In [ ]:
fig = px.line(df_global_confirmed_cases, x="Date", y="Count", color="Country", 
              title='Transition in the number of confirmed cases of each countries',
              template="simple_white")
fig.update_layout(showlegend=False)
fig.show()

### Selected countries

In [ ]:
selected_countries  = ["US", "India", "Spain", "Brazil", "United Kingdom", "Japan"]
df_global_confirmed_cases_selected = df_global_confirmed_cases[df_global_confirmed_cases["Country"].isin(selected_countries)].reset_index(drop=True)

In [ ]:
fig = px.line(df_global_confirmed_cases_selected, x="Date", y="Count", color="Country", 
              title='Transition in the number of confirmed cases of each countries',
              template="simple_white")
fig.update_layout(showlegend=False)
fig.show()

### Check trends

In [ ]:
#Assume a one-week cycle.
freq = 7

df_trends = pd.DataFrame(columns=df_global_confirmed_cases_selected.columns)
for col in selected_countries:
    df_country = df_global_confirmed_cases_selected[df_global_confirmed_cases_selected["Country"] == col]\
                 .reset_index(drop=True).reset_index()
    df_country = df_country[["index", "Count"]]
    res = sm.tsa.seasonal_decompose(df_country.values, freq=freq)
    trend = [item[1] for item in res.trend]
    df_tmp = df_global_confirmed_cases_selected.iloc[df_global_confirmed_cases_selected[df_global_confirmed_cases_selected["Country"] == col].index,:]
    df_tmp["Count"] = trend
    df_trends = pd.concat([df_trends, df_tmp])

In [ ]:
fig = px.line(df_trends, x="Date", y="Count", color="Country", 
              title='Trend component of transition in the number of confirmed cases of each countries',
              template="simple_white")
fig.update_layout(showlegend=False)
fig.show()

# global_death

### tyding

In [ ]:
df_global_deaths = pd.melt(df_global_deaths, id_vars=['Date'], value_vars=country_cols,
                           var_name='Country', value_name='Count')

In [ ]:
df_global_deaths.head()

### Visualize all data

In [ ]:
fig = px.line(df_global_deaths, x="Date", y="Count", color="Country", 
              title='Transition in the number of deaths of each countries',
              template="simple_white")
fig.update_layout(showlegend=False)
fig.show()

### Selected countries

In [ ]:
selected_countries  = ["US", "India", "Spain", "Brazil", "United Kingdom", "Japan"]
df_global_deaths_selected = df_global_deaths[df_global_deaths["Country"].isin(selected_countries)].reset_index(drop=True)

In [ ]:
fig = px.line(df_global_deaths_selected, x="Date", y="Count", color="Country", 
              title='Transition in the number of death of each countries',
              template="simple_white")
fig.update_layout(showlegend=False)
fig.show()

### Check trends

In [ ]:
#Assume a one-week cycle.
freq = 7

df_trends = pd.DataFrame(columns=df_global_deaths_selected.columns)
for col in selected_countries:
    df_country = df_global_deaths_selected[df_global_deaths_selected["Country"] == col]\
                 .reset_index(drop=True).reset_index()
    df_country = df_country[["index", "Count"]]
    res = sm.tsa.seasonal_decompose(df_country.values, freq=freq)
    trend = [item[1] for item in res.trend]
    df_tmp = df_global_deaths_selected.iloc[df_global_deaths_selected[df_global_deaths_selected["Country"] == col].index,:]
    df_tmp["Count"] = trend
    df_trends = pd.concat([df_trends, df_tmp])

In [ ]:
fig = px.line(df_trends, x="Date", y="Count", color="Country", 
              title='Trend component of transition in the number of death of each countries',
              template="simple_white")
fig.update_layout(showlegend=False)
fig.show()